# Aggregations

In [43]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.ml.stat as ml_stat
import pyspark.sql.functions as func
import pyspark.sql.types as types
from pyspark.sql import Window
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('Spark Test App').getOrCreate() 
sc = spark.sparkContext

In [123]:
# Create Data Frame from a csv file with inferred schema
df = spark.read.csv('Auto.csv', header=True, inferSchema=True)
df = df.selectExpr("split(name, ' ')[0] as make", "*")
df.show(5)

+---------+----+---------+------------+----------+------+------------+----+------+--------------------+
|     make| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|
+---------+----+---------+------------+----------+------+------------+----+------+--------------------+
|chevrolet|18.0|        8|       307.0|       130|  3504|        12.0|  70|     1|chevrolet chevell...|
|    buick|15.0|        8|       350.0|       165|  3693|        11.5|  70|     1|   buick skylark 320|
| plymouth|18.0|        8|       318.0|       150|  3436|        11.0|  70|     1|  plymouth satellite|
|      amc|16.0|        8|       304.0|       150|  3433|        12.0|  70|     1|       amc rebel sst|
|     ford|17.0|        8|       302.0|       140|  3449|        10.5|  70|     1|         ford torino|
+---------+----+---------+------------+----------+------+------------+----+------+--------------------+
only showing top 5 rows



### Aggregation Functions

In [18]:
# count
df.selectExpr("count(make)").show()

+-----------+
|count(make)|
+-----------+
|        397|
+-----------+



In [17]:
# count distinct
df.selectExpr("count(distinct make)").show()

+--------------------+
|count(DISTINCT make)|
+--------------------+
|                  37|
+--------------------+



In [20]:
# count for large datasets
df.selectExpr("approx_count_distinct(make)").show()

+---------------------------+
|approx_count_distinct(make)|
+---------------------------+
|                         36|
+---------------------------+



In [22]:
# first and last
df.selectExpr("first(make) as first", "last(make) as last").show()

+---------+-----+
|    first| last|
+---------+-----+
|chevrolet|chevy|
+---------+-----+



In [24]:
# min and max
df.selectExpr("min(mpg)", "max(mpg)").show()

+--------+--------+
|min(mpg)|max(mpg)|
+--------+--------+
|     9.0|    46.6|
+--------+--------+



In [25]:
# sum and average
df.selectExpr("sum(weight)", "avg(weight)").show()

+-----------+------------------+
|sum(weight)|       avg(weight)|
+-----------+------------------+
|    1179194|2970.2619647355164|
+-----------+------------------+



In [28]:
# statistical (population and sampling variance and st dev)
df.selectExpr("var_pop(mpg)", "var_samp(mpg)", "stddev_pop(mpg)", "stddev_samp(mpg)").show()

+------------------+------------------+-----------------+-----------------+
|      var_pop(mpg)|     var_samp(mpg)|  stddev_pop(mpg)| stddev_samp(mpg)|
+------------------+------------------+-----------------+-----------------+
|61.088942128939344|61.243207134315455|7.815941538224256|7.825803928946563|
+------------------+------------------+-----------------+-----------------+



In [29]:
#stat (skewness, kurtosis)
df.selectExpr("skewness(mpg)", "kurtosis(mpg)").show()

+------------------+-------------------+
|     skewness(mpg)|      kurtosis(mpg)|
+------------------+-------------------+
|0.4542802097749803|-0.5259045302021703|
+------------------+-------------------+



In [30]:
# Correlation
df.selectExpr("corr(mpg, horsepower)").show()

+-------------------------------------+
|corr(mpg, CAST(horsepower AS DOUBLE))|
+-------------------------------------+
|                  -0.7784267838977761|
+-------------------------------------+



### Grouping

In [36]:
# Group by
df.groupby('make', 'year').avg('mpg').show(5)

+----------+----+--------+
|      make|year|avg(mpg)|
+----------+----+--------+
|   pontiac|  78|    19.2|
|oldsmobile|  79|   25.35|
|    datsun|  72|    28.0|
|   renault|  77|    36.0|
|      audi|  73|    20.0|
+----------+----+--------+
only showing top 5 rows



In [42]:
# Using aggregate expressions
df.groupby('make').agg(func.expr("avg(mpg) as mean_mpg"), func.expr("stddev(mpg) as stddev_mpg")).show(5)

+--------+------------------+------------------+
|    make|          mean_mpg|        stddev_mpg|
+--------+------------------+------------------+
|   buick|19.182352941176468| 5.760993327339224|
| pontiac|           20.0125| 6.285631763527565|
|mercedes|              25.4|               NaN|
|  toyota|28.372000000000003|5.6594699398441906|
|    saab|              23.9|1.6041612554021285|
+--------+------------------+------------------+
only showing top 5 rows



### Window Functions

In [75]:
# Example: for every make compute average mpg to date
window_spec = Window.partitionBy('make') \
    .orderBy('year') \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

df.groupby('make', 'year') \
    .agg(func.avg('mpg').alias('mean_mpg')) \
    .select('make', 'year', func.avg('mean_mpg').over(window_spec).alias('mean_avg')) \
    .show(5)

+-----+----+------------------+
| make|year|          mean_avg|
+-----+----+------------------+
|buick|  70|              14.5|
|buick|  72|             13.75|
|buick|  73|13.333333333333334|
|buick|  74|             13.25|
|buick|  75|              14.4|
+-----+----+------------------+
only showing top 5 rows



In [97]:
# Ranking 
window_spec = Window.partitionBy('year') \
    .orderBy(func.desc('mpg')) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [100]:
df.groupby('make', 'year') \
    .agg(func.avg('mpg').alias('mpg')) \
    .select('year', 'make', 'mpg', func.dense_rank().over(window_spec).alias('rank')) \
    .show(20)

+----+----------+------------------+----+
|year|      make|               mpg|rank|
+----+----------+------------------+----+
|  78|volkswagen|              37.3|   1|
|  78|     mazda|              32.8|   2|
|  78|     honda|              32.8|   2|
|  78|    datsun|30.166666666666668|   3|
|  78|      ford|            24.875|   4|
|  78|    toyota|              24.3|   5|
|  78| chevrolet|23.233333333333334|   6|
|  78|oldsmobile|             21.85|   7|
|  78|  plymouth|             21.85|   7|
|  78|     dodge|              21.6|   8|
|  78|      saab|              21.6|   8|
|  78|   mercury|              20.5|   9|
|  78|      audi|              20.3|  10|
|  78|   pontiac|              19.2|  11|
|  78|     buick|             19.15|  12|
|  78|       amc|             18.75|  13|
|  78|     volvo|              17.0|  14|
|  78|   peugeot|              16.2|  15|
|  81|   renault|              34.5|   1|
|  81|     honda|34.400000000000006|   2|
+----+----------+-----------------

### Rollup and Cube

In [128]:
df2 = df.where("make in ('audi', 'volkswagen', 'mercedes', 'bmw')") \
       .where('year<72') 
df2.show(5)

+----------+----+---------+------------+----------+------+------------+----+------+--------------------+
|      make| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|
+----------+----+---------+------------+----------+------+------------+----+------+--------------------+
|volkswagen|26.0|        4|        97.0|        46|  1835|        20.5|  70|     2|volkswagen 1131 d...|
|      audi|24.0|        4|       107.0|        90|  2430|        14.5|  70|     2|         audi 100 ls|
|       bmw|26.0|        4|       121.0|       113|  2234|        12.5|  70|     2|            bmw 2002|
|volkswagen|27.0|        4|        97.0|        60|  1834|        19.0|  71|     2|volkswagen model 111|
+----------+----+---------+------------+----------+------+------------+----+------+--------------------+



In [130]:
# Rollup
df2.rollup('make', 'year').avg('mpg').show(20)

+----------+----+--------+
|      make|year|avg(mpg)|
+----------+----+--------+
|volkswagen|  70|    26.0|
|      null|null|   25.75|
|volkswagen|  71|    27.0|
|       bmw|null|    26.0|
|      audi|null|    24.0|
|      audi|  70|    24.0|
|volkswagen|null|    26.5|
|       bmw|  70|    26.0|
+----------+----+--------+



In [134]:
# Cube
df2.cube('make', 'year').avg('mpg').show(20)

+----------+----+------------------+
|      make|year|          avg(mpg)|
+----------+----+------------------+
|      null|  70|25.333333333333332|
|volkswagen|  70|              26.0|
|      null|null|             25.75|
|volkswagen|  71|              27.0|
|       bmw|null|              26.0|
|      null|  71|              27.0|
|      audi|null|              24.0|
|      audi|  70|              24.0|
|volkswagen|null|              26.5|
|       bmw|  70|              26.0|
+----------+----+------------------+

